In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint
from tensorflow import set_random_seed
set_random_seed(12)

Using TensorFlow backend.
C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaco

# Preprocessing

In [2]:
data = pd.read_csv('./datasets/IBM_daily.csv').sort_values(by='date')

In [3]:
data

,date,1. open,2. high,3. low,4. close,5. volume
5217,1999-11-01,98.50,98.8100,96.37,96.75,9551800.0
5216,1999-11-02,96.75,96.8100,93.69,94.81,11105400.0
5215,1999-11-03,95.87,95.9400,93.50,94.37,10369100.0
5214,1999-11-04,94.44,94.4400,90.00,91.56,16697600.0
5213,1999-11-05,92.75,92.9400,90.19,90.25,13737600.0
...,...,...,...,...,...,...
4,2020-07-22,125.90,129.4700,125.80,128.67,8195366.0
3,2020-07-23,129.10,129.3700,127.15,127.33,4220136.0
2,2020-07-24,126.48,127.6459,125.50,125.79,3531076.0
1,2020-07-27,124.86,126.3200,124.71,126.21,3733547.0


In [4]:
data = data.drop(columns='date')
data

,1. open,2. high,3. low,4. close,5. volume
5217,98.50,98.8100,96.37,96.75,9551800.0
5216,96.75,96.8100,93.69,94.81,11105400.0
5215,95.87,95.9400,93.50,94.37,10369100.0
5214,94.44,94.4400,90.00,91.56,16697600.0
5213,92.75,92.9400,90.19,90.25,13737600.0
...,...,...,...,...,...
4,125.90,129.4700,125.80,128.67,8195366.0
3,129.10,129.3700,127.15,127.33,4220136.0
2,126.48,127.6459,125.50,125.79,3531076.0
1,124.86,126.3200,124.71,126.21,3733547.0


In [5]:
# Sequence for the LSTM network
backlook = 92

# Size of data split for test
train_size = 0.8

In [6]:
# Normalise data
normaliser = preprocessing.MinMaxScaler()
data_norm = normaliser.fit_transform(data)

In [7]:
# Normalised chunks
historical_sequences_norm = np.array([data_norm[i : i + backlook].copy() for i in range(len(data_norm) - backlook)])
next_day_open_values_norm = np.array([data_norm[:,0][i + backlook].copy() for i in range(len(data_norm) - backlook)])
next_day_open_values_norm = np.expand_dims(next_day_open_values_norm, -1)

In [8]:
# Y raw data
next_day_open_values = np.array([data.to_numpy()[:,0][i + backlook].copy() for i in range(len(data) - backlook)])
next_day_open_values = np.expand_dims(next_day_open_values, -1)

# Y normaliser
y_normaliser = preprocessing.MinMaxScaler()
y_normaliser.fit_transform(next_day_open_values)

array([[0.3303677 ],
       [0.32881229],
       [0.33814471],
       ...,
       [0.44689853],
       [0.43681951],
       [0.44279226]])

In [9]:
# Train-Test split
split = int(historical_sequences_norm.shape[0] * train_size)

X_train = historical_sequences_norm[:split]
Y_train = next_day_open_values_norm[:split]

X_test = historical_sequences_norm[split:]
Y_test = next_day_open_values_norm[split:]
unscaled_y_test = next_day_open_values[split:]

# Model selection

##### Model constructor

In [10]:
def build_lstm(x, y, batch_size=512, epochs=24, **params):
    
    # List of parameters
    if 'lstmsize' not in params: params['lstmsize'] = x.shape[1]
    if 'density' not in params: params['density'] = int((params['lstmsize']//1.5)*2)
    if 'activation' not in params: params['activation'] = 'relu'
    if 'twice' not in params: params['twice'] = False
    if 'optimizer' not in params: params['optimizer'] = 'adam'
    if 'shuffle' not in params: params['shuffle'] = False
    
    # Model definition
    model = Sequential()
    
    model.add(LSTM(params['lstmsize'], input_shape=x.shape[1:], return_sequences=params['twice']))
    
    if 'dropout' in params:
        model.add(Dropout(params['dropout']))
    
    if params['twice']:
        model.add(LSTM(params['lstmsize']))
        
        if 'dropout' in params:
            model.add(Dropout(params['dropout']))
            
    model.add(Dense(params['density'], activation=params['activation']))
    
    if 'full_density' in params and params['full_density']:
        density = params['density']//2
        while density >= 12:
            model.add(Dense(density, activation=params['activation']))
            density //= 2
            
    model.add(Dense(1, activation='linear'))
    
    model.compile(loss='mse', optimizer=params['optimizer'])
    
    if 'callbacks' in params:
        callback = model.fit(x=x, y=y, validation_split=0.1, batch_size=batch_size, epochs=epochs, shuffle=params['shuffle'],
                            callbacks=params['callbacks'])
    else:
        callback = model.fit(x=x, y=y, validation_split=0.1, batch_size=batch_size, epochs=epochs, shuffle=params['shuffle'])
    
    return [model, callback, params]

##### Evolutive algorith to search for the most optimal model

In [11]:
def set_genes(x, y, population_size, population=[]):
    if population_size-len(population) < 0:
        print('population size must be greater or equal than actual population')
        return
    
    if len(population) > 0:
        if len(population) < 3:
            print('population should be empty or 3 at least')
            return
        
        population[2] = breed_genes(x, y, population[0][2])
        population[1] = breed_genes(x, y, combine=(population[0][2], population[1][2]))
        population = population[:3]
    
    for _ in range(population_size-len(population)):
        subject = breed_genes(x, y)
        population.append(subject)
        
    return population

def breed_genes(x, y, genes={}, combine=None):
    genes['x'] = x
    genes['y'] = y
    
    if type(combine) is list or type(combine) is tuple:
        genes['lstmsize'] = combine[np.random.randint(0,2)]['lstmsize']
        genes['density'] = combine[np.random.randint(0,2)]['density']
        genes['activation'] = combine[np.random.randint(0,2)]['activation']
        genes['twice'] = combine[np.random.randint(0,2)]['twice']
        genes['optimizer'] = combine[np.random.randint(0,2)]['optimizer']
        genes['shuffle'] = combine[np.random.randint(0,2)]['shuffle']
        
        genes['dropout'] = combine[np.random.randint(0,2)].get('dropout')
        if genes['dropout'] is None: del genes['dropout']
        genes['full_density'] = combine[np.random.randint(0,2)].get('full_density')
        if genes['full_density'] is None: del genes['full_density']
            
    else:
        if np.random.randint(0,3) == 1:
            genes['lstmsize'] = int((np.random.randint(x.shape[1],x.shape[1]*2)//2)*2)

        if np.random.randint(0,3) == 1:
            genes['density'] = int((np.random.randint(x.shape[1]//2,x.shape[1]*2.66)//2)*2)

        if np.random.randint(0,3) == 1:
            activation = [
                'relu',
                'sigmoid',
                'softplus',
                'softsign',
                'tanh',
                'selu',
                'elu',
                'exponential'
            ]
            genes['activation'] = activation[np.random.randint(0,8)]

        if np.random.randint(0,3) == 1:
            genes['twice'] = True

        if np.random.randint(0,3) == 1:
            optimizer = [
                'sgd',
                'rmsprop',
                'adam',
                'adadelta',
                'adagrad',
                'adamax',
                'nadam'
            ]
            genes['optimizer'] = optimizer[np.random.randint(0,7)]

        if np.random.randint(0,3) == 1:
            genes['shuffle'] = True

        if np.random.randint(0,3) == 1:
            genes['dropout'] = np.random.randint(1,4)/10

        if np.random.randint(0,3) == 1:
            genes['full_density'] = True
            
    new_model = build_lstm(**genes)
        
    return new_model

In [12]:
def breed_population(x, y, generations, population_size, population=[]):
    
    if generations > 1 and population_size < 3:
        print('population size should be of a minimum of 3 for more than one generation')
        return
    
    for g in range(generations):
        print(f'\nGENERATION {g}\n')
        population = set_genes(x, y, population_size, population)
        population = sorted(population, key=lambda x: x[1].history['val_loss'][-1]+x[1].history['loss'][-1])
        
    return population

In [13]:
best_models = breed_population(X_train, Y_train, generations=20, population_size=10)


GENERATION 0

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 3690 samples, validate on 410 samples
Epoch 1/24
3690/3690 [==============================] - 2s 636us/step - loss: 0.2375 - val_loss: 0.0765
Epoch 2/24
3690/3690 [==============================] - 1s 312us/step - loss: 0.0699 - val_loss: 0.0227
Epoch 3/24
3690/3690 [==============================] - 1s 312us/step - loss: 0.0691 - val_loss: 0.0354
Epoch 4/24
3690/3690 [==============================] - 1s 319us/step - loss: 0.0661 - val_loss: 0.0362
Epoch 5/24
3690/3690 [==============================] - 1s 318us/step - loss: 0.0662 - val_loss: 0.0278
Epoch 6/24
3690/3690 [==============================] - 1s 311us/step - loss: 0.0645 - val_loss: 0.0376
Epoch 7/24
3690/3690 [==============================] - 1s 313us/step - loss: 0.0681 - val_loss: 0.0346
Epoch 8/24
3690/3690 [==============================] - 1s 310us/step - loss: 0.0676 - val

3690/3690 [==============================] - 1s 323us/step - loss: 0.0035 - val_loss: 0.0463
Train on 3690 samples, validate on 410 samples
Epoch 1/24
3690/3690 [==============================] - 2s 565us/step - loss: nan - val_loss: nan
Epoch 2/24
3690/3690 [==============================] - 1s 321us/step - loss: nan - val_loss: nan
Epoch 3/24
3690/3690 [==============================] - 1s 320us/step - loss: nan - val_loss: nan
Epoch 4/24
3690/3690 [==============================] - 1s 320us/step - loss: nan - val_loss: nan
Epoch 5/24
3690/3690 [==============================] - 1s 321us/step - loss: nan - val_loss: nan
Epoch 6/24
3690/3690 [==============================] - 1s 321us/step - loss: nan - val_loss: nan
Epoch 7/24
3690/3690 [==============================] - 1s 322us/step - loss: nan - val_loss: nan
Epoch 8/24
3690/3690 [==============================] - 1s 336us/step - loss: nan - val_loss: nan
Epoch 9/24
3690/3690 [==============================] - 1s 321us/step - loss

3690/3690 [==============================] - 1s 362us/step - loss: 0.0023 - val_loss: 7.7064e-04
Epoch 5/24
3690/3690 [==============================] - 1s 364us/step - loss: 0.0018 - val_loss: 0.0026
Epoch 6/24
3690/3690 [==============================] - 1s 362us/step - loss: 0.0015 - val_loss: 5.5479e-04
Epoch 7/24
3690/3690 [==============================] - 1s 362us/step - loss: 0.0014 - val_loss: 0.0011
Epoch 8/24
3690/3690 [==============================] - 1s 368us/step - loss: 0.0013 - val_loss: 9.2452e-04
Epoch 9/24
3690/3690 [==============================] - 1s 366us/step - loss: 0.0012 - val_loss: 9.6949e-04
Epoch 10/24
3690/3690 [==============================] - 1s 364us/step - loss: 0.0012 - val_loss: 9.2587e-04
Epoch 11/24
3690/3690 [==============================] - 1s 361us/step - loss: 0.0012 - val_loss: 6.4200e-04
Epoch 12/24
3690/3690 [==============================] - 1s 361us/step - loss: 0.0011 - val_loss: 6.0448e-04
Epoch 13/24
3690/3690 [=====================

3690/3690 [==============================] - 1s 327us/step - loss: 0.0591 - val_loss: 0.0302
Epoch 8/24
3690/3690 [==============================] - 1s 326us/step - loss: 0.0571 - val_loss: 0.0279
Epoch 9/24
3690/3690 [==============================] - 1s 324us/step - loss: 0.0553 - val_loss: 0.0259
Epoch 10/24
3690/3690 [==============================] - 1s 327us/step - loss: 0.0537 - val_loss: 0.0244
Epoch 11/24
3690/3690 [==============================] - 1s 326us/step - loss: 0.0520 - val_loss: 0.0236
Epoch 12/24
3690/3690 [==============================] - 1s 327us/step - loss: 0.0509 - val_loss: 0.0225
Epoch 13/24
3690/3690 [==============================] - 1s 327us/step - loss: 0.0492 - val_loss: 0.0215
Epoch 14/24
3690/3690 [==============================] - 1s 326us/step - loss: 0.0475 - val_loss: 0.0196
Epoch 15/24
3690/3690 [==============================] - 1s 326us/step - loss: 0.0460 - val_loss: 0.0185
Epoch 16/24
3690/3690 [==============================] - 1s 328us/ste

3690/3690 [==============================] - 2s 569us/step - loss: 0.0682 - val_loss: 0.0390
Epoch 11/24
3690/3690 [==============================] - 2s 569us/step - loss: 0.0681 - val_loss: 0.0365
Epoch 12/24
3690/3690 [==============================] - 2s 571us/step - loss: 0.0682 - val_loss: 0.0350
Epoch 13/24
3690/3690 [==============================] - 2s 573us/step - loss: 0.0682 - val_loss: 0.0360
Epoch 14/24
3690/3690 [==============================] - 2s 569us/step - loss: 0.0681 - val_loss: 0.0385
Epoch 15/24
3690/3690 [==============================] - 2s 568us/step - loss: 0.0681 - val_loss: 0.0393
Epoch 16/24
3690/3690 [==============================] - 2s 573us/step - loss: 0.0682 - val_loss: 0.0387
Epoch 17/24
3690/3690 [==============================] - 2s 569us/step - loss: 0.0682 - val_loss: 0.0369
Epoch 18/24
3690/3690 [==============================] - 2s 569us/step - loss: 0.0681 - val_loss: 0.0357
Epoch 19/24
3690/3690 [==============================] - 2s 569us/s

...

3690/3690 [==============================] - 2s 572us/step - loss: 0.0021 - val_loss: 0.0030
Train on 3690 samples, validate on 410 samples
Epoch 1/24
3690/3690 [==============================] - 43s 12ms/step - loss: 0.1148 - val_loss: 0.0016
Epoch 2/24
3690/3690 [==============================] - 2s 567us/step - loss: 0.0066 - val_loss: 0.0062
Epoch 3/24
3690/3690 [==============================] - 2s 578us/step - loss: 0.0060 - val_loss: 0.0035
Epoch 4/24
3690/3690 [==============================] - 2s 563us/step - loss: 0.0061 - val_loss: 0.0011
Epoch 5/24
3690/3690 [==============================] - 2s 571us/step - loss: 0.0025 - val_loss: 0.0019
Epoch 6/24
3690/3690 [==============================] - 2s 566us/step - loss: 0.0053 - val_loss: 0.0124
Epoch 7/24
3690/3690 [==============================] - 2s 572us/step - loss: 0.0059 - val_loss: 9.0003e-04
Epoch 8/24
3690/3690 [==============================] - 2s 571us/step - loss: 0.0021 - val_loss: 0.0017
Epoch 9/24
3690/3690 [==

3690/3690 [==============================] - 2s 573us/step - loss: 0.0169 - val_loss: 0.0098
Epoch 5/24
3690/3690 [==============================] - 2s 569us/step - loss: 0.0080 - val_loss: 0.0212
Epoch 6/24
3690/3690 [==============================] - 2s 568us/step - loss: 0.0045 - val_loss: 0.0164
Epoch 7/24
3690/3690 [==============================] - 2s 572us/step - loss: 0.0029 - val_loss: 0.0159
Epoch 8/24
3690/3690 [==============================] - 2s 576us/step - loss: 0.0025 - val_loss: 0.0105
Epoch 9/24
3690/3690 [==============================] - 2s 571us/step - loss: 0.0025 - val_loss: 0.0071
Epoch 10/24
3690/3690 [==============================] - 2s 565us/step - loss: 0.0023 - val_loss: 0.0067
Epoch 11/24
3690/3690 [==============================] - 2s 561us/step - loss: 0.0023 - val_loss: 0.0042
Epoch 12/24
3690/3690 [==============================] - 2s 565us/step - loss: 0.0030 - val_loss: 0.0021
Epoch 13/24
3690/3690 [==============================] - 2s 561us/step -

3690/3690 [==============================] - 2s 619us/step - loss: 0.0012 - val_loss: 0.0016
Epoch 8/24
3690/3690 [==============================] - 2s 609us/step - loss: 0.0011 - val_loss: 7.1944e-04
Epoch 9/24
3690/3690 [==============================] - 2s 603us/step - loss: 0.0010 - val_loss: 7.6588e-04
Epoch 10/24
3690/3690 [==============================] - 2s 614us/step - loss: 9.8701e-04 - val_loss: 7.4881e-04
Epoch 11/24
3690/3690 [==============================] - 2s 605us/step - loss: 9.6130e-04 - val_loss: 7.3342e-04
Epoch 12/24
3690/3690 [==============================] - 2s 605us/step - loss: 9.4591e-04 - val_loss: 7.2516e-04
Epoch 13/24
3690/3690 [==============================] - 2s 604us/step - loss: 9.3436e-04 - val_loss: 5.3631e-04
Epoch 14/24
3690/3690 [==============================] - 2s 610us/step - loss: 8.7709e-04 - val_loss: 5.9098e-04
Epoch 15/24
3690/3690 [==============================] - 2s 606us/step - loss: 9.1575e-04 - val_loss: 5.6750e-04
Epoch 16/24
3

Epoch 11/24
3690/3690 [==============================] - 2s 609us/step - loss: 0.0117 - val_loss: 0.0164
Epoch 12/24
3690/3690 [==============================] - 2s 605us/step - loss: 0.0168 - val_loss: 0.0034
Epoch 13/24
3690/3690 [==============================] - 2s 603us/step - loss: 0.0107 - val_loss: 0.0063
Epoch 14/24
3690/3690 [==============================] - 2s 601us/step - loss: 0.0117 - val_loss: 0.0035
Epoch 15/24
3690/3690 [==============================] - 2s 605us/step - loss: 0.0116 - val_loss: 0.0015
Epoch 16/24
3690/3690 [==============================] - 2s 603us/step - loss: 0.0130 - val_loss: 0.0037
Epoch 17/24
3690/3690 [==============================] - 2s 606us/step - loss: 0.0092 - val_loss: 0.0025
Epoch 18/24
3690/3690 [==============================] - 2s 611us/step - loss: 0.0096 - val_loss: 0.0015
Epoch 19/24
3690/3690 [==============================] - 2s 609us/step - loss: 0.0091 - val_loss: 0.0073
Epoch 20/24
3690/3690 [==============================] 

3690/3690 [==============================] - 2s 660us/step - loss: 0.0020 - val_loss: 0.0055
Epoch 16/24
3690/3690 [==============================] - 2s 649us/step - loss: 0.0022 - val_loss: 0.0031
Epoch 17/24
3690/3690 [==============================] - 2s 654us/step - loss: 0.0019 - val_loss: 0.0020
Epoch 18/24
3690/3690 [==============================] - 2s 653us/step - loss: 0.0015 - val_loss: 0.0085
Epoch 19/24
3690/3690 [==============================] - 2s 645us/step - loss: 0.0019 - val_loss: 0.0021
Epoch 20/24
3690/3690 [==============================] - 2s 656us/step - loss: 0.0013 - val_loss: 6.7875e-04
Epoch 21/24
3690/3690 [==============================] - 2s 652us/step - loss: 0.0013 - val_loss: 0.0012
Epoch 22/24
3690/3690 [==============================] - 2s 656us/step - loss: 0.0017 - val_loss: 8.4776e-04
Epoch 23/24
3690/3690 [==============================] - 2s 652us/step - loss: 0.0018 - val_loss: 0.0011
Epoch 24/24
3690/3690 [==============================] - 2s

In [14]:
best_models[0][1].history['val_loss']

[0.0037763554137200117,
 0.0012234121095389128,
 0.003760388819500804,
 0.0006878753774799407,
 0.0023324117064476013,
 0.00045704448712058365,
 0.0007037113537080586,
 0.0008244329364970326,
 0.0004389943496789783,
 0.0005706932861357927,
 0.0006469497457146645,
 0.0006032950477674603,
 0.0005997797125019133,
 0.00046004957403056324,
 0.0003956618020310998,
 0.0004812253755517304,
 0.0003788948815781623,
 0.0003888547362294048,
 0.00037597835762426257,
 0.0003765215224120766,
 0.0004003244685009122,
 0.0003607597027439624,
 0.0003644057142082602,
 0.0003927744401153177]

In [15]:
print('optimal parameters')
print('_______________________')
for key,val in best_models[0][2].items():
    if key != 'x' and key != 'y': print(f'{key}: {val}')

optimal parameters
_______________________
density: 46
activation: tanh
twice: True
optimizer: adam
shuffle: True
full_density: True
dropout: 0.1
lstmsize: 150


In [16]:
best_models[0][0].summary()

Model: "sequential_172"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_343 (LSTM)              (None, 92, 150)           93600     
_________________________________________________________________
dropout_339 (Dropout)        (None, 92, 150)           0         
_________________________________________________________________
lstm_344 (LSTM)              (None, 150)               180600    
_________________________________________________________________
dropout_340 (Dropout)        (None, 150)               0         
_________________________________________________________________
dense_861 (Dense)            (None, 46)                6946      
_________________________________________________________________
dense_862 (Dense)            (None, 23)                1081      
_________________________________________________________________
dense_863 (Dense)            (None, 1)              

In [17]:
ibm_checkpoint = ModelCheckpoint(
    filepath='./checkpoints/IBM.{epoch}-{val_loss}.hdf5',
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True
)

In [18]:
params = best_models[0][2]
params['x'] = X_train
params['y'] = Y_train
params['epochs'] = 2000
params['callbacks'] = [ibm_checkpoint]

In [19]:
final_model = build_lstm(**params)

Train on 3690 samples, validate on 410 samples
Epoch 1/2000
3690/3690 [==============================] - 51s 14ms/step - loss: 0.0462 - val_loss: 0.0279
Epoch 2/2000
3690/3690 [==============================] - 2s 622us/step - loss: 0.0095 - val_loss: 0.0034
Epoch 3/2000
3690/3690 [==============================] - 2s 615us/step - loss: 0.0037 - val_loss: 5.0629e-04
Epoch 4/2000
3690/3690 [==============================] - 2s 611us/step - loss: 0.0021 - val_loss: 5.5345e-04
Epoch 5/2000
3690/3690 [==============================] - 2s 616us/step - loss: 0.0016 - val_loss: 0.0020
Epoch 6/2000
3690/3690 [==============================] - 2s 612us/step - loss: 0.0013 - val_loss: 4.4862e-04
Epoch 7/2000
3690/3690 [==============================] - 2s 612us/step - loss: 0.0012 - val_loss: 0.0016
Epoch 8/2000
3690/3690 [==============================] - 2s 614us/step - loss: 0.0011 - val_loss: 5.0256e-04
Epoch 9/2000
3690/3690 [==============================] - 2s 616us/step - loss: 9.9473e-0

3690/3690 [==============================] - 2s 618us/step - loss: 4.8694e-04 - val_loss: 4.6213e-04
Epoch 73/2000
3690/3690 [==============================] - 2s 623us/step - loss: 4.6847e-04 - val_loss: 3.0941e-04
Epoch 74/2000
3690/3690 [==============================] - 2s 613us/step - loss: 5.1310e-04 - val_loss: 3.1288e-04
Epoch 75/2000
3690/3690 [==============================] - 2s 614us/step - loss: 5.1325e-04 - val_loss: 6.4923e-04
Epoch 76/2000
3690/3690 [==============================] - 2s 614us/step - loss: 4.5610e-04 - val_loss: 4.5200e-04
Epoch 77/2000
3690/3690 [==============================] - 2s 617us/step - loss: 4.6606e-04 - val_loss: 3.1748e-04
Epoch 78/2000
3690/3690 [==============================] - 2s 614us/step - loss: 4.6991e-04 - val_loss: 6.8803e-04
Epoch 79/2000
3690/3690 [==============================] - 2s 617us/step - loss: 4.8112e-04 - val_loss: 3.1360e-04
Epoch 80/2000
3690/3690 [==============================] - 2s 615us/step - loss: 5.0844e-04 - 

Epoch 143/2000
3690/3690 [==============================] - 2s 607us/step - loss: 3.2285e-04 - val_loss: 2.6399e-04
Epoch 144/2000
3690/3690 [==============================] - 2s 607us/step - loss: 3.2825e-04 - val_loss: 2.7636e-04
Epoch 145/2000
3690/3690 [==============================] - 2s 602us/step - loss: 3.2532e-04 - val_loss: 3.3646e-04
Epoch 146/2000
3690/3690 [==============================] - 2s 604us/step - loss: 3.4267e-04 - val_loss: 2.9362e-04
Epoch 147/2000
3690/3690 [==============================] - 2s 612us/step - loss: 3.2843e-04 - val_loss: 2.4187e-04
Epoch 148/2000
3690/3690 [==============================] - 2s 605us/step - loss: 3.1947e-04 - val_loss: 2.2470e-04
Epoch 149/2000
3690/3690 [==============================] - 2s 606us/step - loss: 3.2632e-04 - val_loss: 2.2888e-04
Epoch 150/2000
3690/3690 [==============================] - 2s 607us/step - loss: 3.0913e-04 - val_loss: 2.1856e-04
Epoch 151/2000
3690/3690 [==============================] - 2s 597us/ste

3690/3690 [==============================] - 2s 618us/step - loss: 2.3378e-04 - val_loss: 1.6369e-04
Epoch 214/2000
3690/3690 [==============================] - 2s 611us/step - loss: 2.4731e-04 - val_loss: 1.6848e-04
Epoch 215/2000
3690/3690 [==============================] - 2s 616us/step - loss: 2.4495e-04 - val_loss: 1.5762e-04
Epoch 216/2000
3690/3690 [==============================] - 2s 612us/step - loss: 2.2977e-04 - val_loss: 2.8125e-04
Epoch 217/2000
3690/3690 [==============================] - 2s 614us/step - loss: 2.5264e-04 - val_loss: 3.4826e-04
Epoch 218/2000
3690/3690 [==============================] - 2s 617us/step - loss: 2.4793e-04 - val_loss: 2.8786e-04
Epoch 219/2000
3690/3690 [==============================] - 2s 615us/step - loss: 2.4440e-04 - val_loss: 3.1867e-04
Epoch 220/2000
3690/3690 [==============================] - 2s 621us/step - loss: 2.6046e-04 - val_loss: 1.8658e-04
Epoch 221/2000
3690/3690 [==============================] - 2s 614us/step - loss: 2.282

Epoch 284/2000
3690/3690 [==============================] - 2s 613us/step - loss: 1.8105e-04 - val_loss: 1.5648e-04
Epoch 285/2000
3690/3690 [==============================] - 2s 618us/step - loss: 1.8426e-04 - val_loss: 1.1098e-04
Epoch 286/2000
3690/3690 [==============================] - 2s 615us/step - loss: 1.7818e-04 - val_loss: 1.0888e-04
Epoch 287/2000
3690/3690 [==============================] - 2s 611us/step - loss: 1.8167e-04 - val_loss: 1.2163e-04
Epoch 288/2000
3690/3690 [==============================] - 2s 614us/step - loss: 1.6721e-04 - val_loss: 1.3025e-04
Epoch 289/2000
3690/3690 [==============================] - 2s 612us/step - loss: 1.9936e-04 - val_loss: 1.8876e-04
Epoch 290/2000
3690/3690 [==============================] - 2s 618us/step - loss: 2.0321e-04 - val_loss: 3.5028e-04
Epoch 291/2000
3690/3690 [==============================] - 2s 613us/step - loss: 2.1984e-04 - val_loss: 1.2265e-04
Epoch 292/2000
3690/3690 [==============================] - 2s 621us/ste

...

3690/3690 [==============================] - 2s 616us/step - loss: 7.7491e-05 - val_loss: 1.0830e-04
Epoch 1690/2000
3690/3690 [==============================] - 2s 612us/step - loss: 6.8769e-05 - val_loss: 7.7777e-05
Epoch 1691/2000
3690/3690 [==============================] - 2s 621us/step - loss: 6.6054e-05 - val_loss: 1.2168e-04
Epoch 1692/2000
3690/3690 [==============================] - 2s 614us/step - loss: 7.2893e-05 - val_loss: 1.1028e-04
Epoch 1693/2000
3690/3690 [==============================] - 2s 607us/step - loss: 7.2134e-05 - val_loss: 1.1449e-04
Epoch 1694/2000
3690/3690 [==============================] - 2s 610us/step - loss: 6.9538e-05 - val_loss: 1.5928e-04
Epoch 1695/2000
3690/3690 [==============================] - 2s 617us/step - loss: 7.1016e-05 - val_loss: 8.0295e-05
Epoch 1696/2000
3690/3690 [==============================] - 2s 618us/step - loss: 6.9281e-05 - val_loss: 1.3544e-04
Epoch 1697/2000
3690/3690 [==============================] - 2s 612us/step - los

3690/3690 [==============================] - 2s 620us/step - loss: 6.5579e-05 - val_loss: 9.5203e-05
Epoch 1760/2000
3690/3690 [==============================] - 2s 623us/step - loss: 7.1106e-05 - val_loss: 1.1116e-04
Epoch 1761/2000
3690/3690 [==============================] - 2s 613us/step - loss: 9.0443e-05 - val_loss: 7.7151e-05
Epoch 1762/2000
3690/3690 [==============================] - 2s 612us/step - loss: 7.7594e-05 - val_loss: 1.6993e-04
Epoch 1763/2000
3690/3690 [==============================] - 2s 612us/step - loss: 7.1252e-05 - val_loss: 1.1660e-04
Epoch 1764/2000
3690/3690 [==============================] - 2s 622us/step - loss: 6.9694e-05 - val_loss: 1.0116e-04
Epoch 1765/2000
3690/3690 [==============================] - 2s 617us/step - loss: 7.8380e-05 - val_loss: 9.6487e-05
Epoch 1766/2000
3690/3690 [==============================] - 2s 617us/step - loss: 6.9380e-05 - val_loss: 1.3131e-04
Epoch 1767/2000
3690/3690 [==============================] - 2s 621us/step - los

3690/3690 [==============================] - 2s 611us/step - loss: 6.7034e-05 - val_loss: 1.4799e-04
Epoch 1830/2000
3690/3690 [==============================] - 2s 617us/step - loss: 7.0611e-05 - val_loss: 1.5520e-04
Epoch 1831/2000
3690/3690 [==============================] - 2s 615us/step - loss: 6.6209e-05 - val_loss: 1.4383e-04
Epoch 1832/2000
3690/3690 [==============================] - 2s 618us/step - loss: 7.1554e-05 - val_loss: 9.1688e-05
Epoch 1833/2000
3690/3690 [==============================] - 2s 616us/step - loss: 7.2773e-05 - val_loss: 1.0893e-04
Epoch 1834/2000
3690/3690 [==============================] - 2s 614us/step - loss: 6.9924e-05 - val_loss: 1.2326e-04
Epoch 1835/2000
3690/3690 [==============================] - 2s 615us/step - loss: 7.8138e-05 - val_loss: 1.1462e-04
Epoch 1836/2000
3690/3690 [==============================] - 2s 612us/step - loss: 7.2636e-05 - val_loss: 1.5118e-04
Epoch 1837/2000
3690/3690 [==============================] - 2s 620us/step - los

3690/3690 [==============================] - 2s 615us/step - loss: 6.6853e-05 - val_loss: 1.2284e-04
Epoch 1900/2000
3690/3690 [==============================] - 2s 618us/step - loss: 6.6107e-05 - val_loss: 1.0327e-04
Epoch 1901/2000
3690/3690 [==============================] - 2s 615us/step - loss: 6.9034e-05 - val_loss: 9.4328e-05
Epoch 1902/2000
3690/3690 [==============================] - 2s 614us/step - loss: 6.9684e-05 - val_loss: 1.5862e-04
Epoch 1903/2000
3690/3690 [==============================] - 2s 614us/step - loss: 6.7158e-05 - val_loss: 1.4230e-04
Epoch 1904/2000
3690/3690 [==============================] - 2s 612us/step - loss: 6.7312e-05 - val_loss: 1.6107e-04
Epoch 1905/2000
3690/3690 [==============================] - 2s 614us/step - loss: 7.1510e-05 - val_loss: 1.3009e-04
Epoch 1906/2000
3690/3690 [==============================] - 2s 617us/step - loss: 8.2057e-05 - val_loss: 9.7104e-05
Epoch 1907/2000
3690/3690 [==============================] - 2s 618us/step - los

3690/3690 [==============================] - 2s 613us/step - loss: 6.6160e-05 - val_loss: 1.1695e-04
Epoch 1970/2000
3690/3690 [==============================] - 2s 603us/step - loss: 6.7931e-05 - val_loss: 1.2352e-04
Epoch 1971/2000
3690/3690 [==============================] - 2s 620us/step - loss: 7.0927e-05 - val_loss: 1.4633e-04
Epoch 1972/2000
3690/3690 [==============================] - 2s 619us/step - loss: 7.6363e-05 - val_loss: 1.2958e-04
Epoch 1973/2000
3690/3690 [==============================] - 2s 616us/step - loss: 7.4942e-05 - val_loss: 3.2709e-04
Epoch 1974/2000
3690/3690 [==============================] - 2s 616us/step - loss: 7.3097e-05 - val_loss: 2.2245e-04
Epoch 1975/2000
3690/3690 [==============================] - 2s 622us/step - loss: 7.6277e-05 - val_loss: 1.8148e-04
Epoch 1976/2000
3690/3690 [==============================] - 2s 618us/step - loss: 7.5089e-05 - val_loss: 1.2009e-04
Epoch 1977/2000
3690/3690 [==============================] - 2s 615us/step - los

In [20]:
final_model[2]

{'density': 46,
 'activation': 'tanh',
 'twice': True,
 'optimizer': 'adam',
 'shuffle': True,
 'full_density': True,
 'dropout': 0.1,
 'lstmsize': 150,
 'callbacks': [<keras.callbacks.callbacks.ModelCheckpoint at 0x1d02448cfc8>]}

In [21]:
final_model[0].summary()

Model: "sequential_182"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_363 (LSTM)              (None, 92, 150)           93600     
_________________________________________________________________
dropout_359 (Dropout)        (None, 92, 150)           0         
_________________________________________________________________
lstm_364 (LSTM)              (None, 150)               180600    
_________________________________________________________________
dropout_360 (Dropout)        (None, 150)               0         
_________________________________________________________________
dense_896 (Dense)            (None, 46)                6946      
_________________________________________________________________
dense_897 (Dense)            (None, 23)                1081      
_________________________________________________________________
dense_898 (Dense)            (None, 1)              

In [25]:
final_model[0].load_weights('./checkpoints/IBM_MSE/IBM.(actual_best).hdf5')
y_predicted = final_model[0].predict(X_test)

In [26]:
true_y_test = y_normaliser.inverse_transform(Y_test)
predicted_y_test = y_normaliser.inverse_transform(y_predicted)

In [60]:
loss = np.mean((true_y_test - predicted_y_test)**2)
medium_error = np.mean(abs(true_y_test - predicted_y_test))
print(f'MSE is {loss:.2f}\nMedium error is {medium_error:.2f}')

MSE is 1.99
Medium error is 0.81


In [44]:
up_down_test = []
up_down_predicted = []

for y,x in zip(Y_test,X_test):
    if y[0] > x[-1][0]: up_down_test.append(1)
    elif y[0] < x[-1][0]: up_down_test.append(-1)
    else: up_down_test.append(0)
        
for y,x in zip(y_predicted,X_test):
    if y[0] > x[-1][0]: up_down_predicted.append(1)
    elif y[0] < x[-1][0]: up_down_predicted.append(-1)
    else: up_down_predicted.append(0)

In [54]:
print(f'Trend accuracy is: {(sum(1 for r,p in zip(up_down_test,up_down_predicted) if r == p)/len(up_down_test))*100:.2f}%')

Trend accuracy is: 81.38%


In [63]:
print(f'Accuracy for growing trend is: {(sum(1 for r,p in zip(up_down_test,up_down_predicted) if r == 1 and p == 1)/sum(1 for v in up_down_test if v == 1))*100:.2f}%')
print(f'Accuracy for lowing trend is: {(sum(1 for r,p in zip(up_down_test,up_down_predicted) if r == -1 and p == -1)/sum(1 for v in up_down_test if v == -1))*100:.2f}%')

Accuracy for growing trend is: 74.71%
Accuracy for lowing trend is: 89.02%
